In [6]:
from selenium import webdriver
import requests
from time import sleep
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import selenium
import time
import pandas as pd
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [33]:
soup = BeautifulSoup(r.text, 'html.parser')
this = soup.find(id="ctl00_ctl00_ContentSection_ContentPlaceHolder1_RecordViewDetails_rptDetailsView_ctl00_moreDetails_copySmiles")
this.

<bound method PageElement.extract of <button class="clipboard-copy-button hide-below-tablet" id="ctl00_ctl00_ContentSection_ContentPlaceHolder1_RecordViewDetails_rptDetailsView_ctl00_moreDetails_copySmiles" onmouseover="ClipboardCopyInit(this, 'C(CCCCCCCC[P+](CCCCCC)(CCCCCC)CCCCCC)CCCCC', 'SMILES', 'ctl00_ctl00_ContentSection_ContentPlaceHolder1_RecordViewDetails_rptDetailsView_ctl00_moreDetails_copySmiles_status'); return false;">Copy</button>>

In [9]:
paper_url = 'http://ilthermo.boulder.nist.gov/ILT2/ilsearch?cmp=&ncmp=0&year=&auth=&keyw=&prp=blXM'
r = requests.get(paper_url)

In [23]:
data

[[['283.09'], ['101.325'], ['0.976', '0.067']],
 [['283.09'], ['101.325'], ['0.977', '0.067']],
 [['283.1'], ['101.325'], ['0.99', '0.068']],
 [['283.1'], ['101.325'], ['0.991', '0.069']],
 [['283.1'], ['101.325'], ['0.993', '0.069']],
 [['288.13'], ['101.325'], ['0.693', '0.045']],
 [['288.13'], ['101.325'], ['0.694', '0.045']],
 [['288.13'], ['101.325'], ['0.698', '0.045']],
 [['288.13'], ['101.325'], ['0.699', '0.045']],
 [['293.07'], ['101.325'], ['0.503', '0.03']],
 [['293.07'], ['101.325'], ['0.503', '0.03']],
 [['293.07'], ['101.325'], ['0.505', '0.03']],
 [['293.07'], ['101.325'], ['0.505', '0.03']],
 [['293.07'], ['101.325'], ['0.506', '0.031']],
 [['298.15'], ['101.325'], ['0.336', '0.019']],
 [['298.15'], ['101.325'], ['0.337', '0.019']],
 [['303.15'], ['101.325'], ['0.256', '0.013']],
 [['303.15'], ['101.325'], ['0.256', '0.013']],
 [['303.15'], ['101.325'], ['0.256', '0.013']],
 [['303.15'], ['101.325'], ['0.256', '0.013']],
 [['303.15'], ['101.325'], ['0.256', '0.013']],


In [17]:
paper_url = "http://ilthermo.boulder.nist.gov/ILT2/ilsearch?"\
    "cmp=&ncmp=1&year=&auth=&keyw=&prp=blXM"
r = requests.get(paper_url)
header = r.json()['header']
papers = r.json()['res']

data_url = 'http://ilthermo.boulder.nist.gov/ILT2/ilset?set={paper_id}'
for paper in papers[:1]:
    r = requests.get(data_url.format(paper_id=paper[0]))
    data = r.json()['data']
    print(paper)
    #then do whatever you want to data like writing to a file
    sleep(0.5) #import step to avoid getting banned by server

['CDkVL', 'Ferreira et al. (2012a)', 'Viscosity', 'Liquid', 'ADWgvf', None, None, '159', 'trihexyl(tetradecyl)phosphonium tris(pentafluoroethyl)trifluorophosphate']


In [11]:
###a hacky hack solution to cleaning raw ILThermo data
df = pd.read_csv("../salty/data/viscosity_full.csv")
uniqueSalts = pd.DataFrame(df["salt_name"].unique(), columns=["salts"])
salts = uniqueSalts
anions=[]
cations=[]
for i in range(salts.shape[0]):
    if len(salts['salts'].iloc[i].split()) == 2:
        cations.append(salts['salts'].iloc[i].split()[0])
        anions.append(salts['salts'].iloc[i].split()[1])
    elif len(salts['salts'].iloc[i].split()) == 3:
        #two word cation
        if"tris(2-hydroxyethyl) methylammonium" in salts['salts'].iloc[i]:
            first = salts['salts'].iloc[i].split()[0]
            second = salts['salts'].iloc[i].split()[1]
            anions.append(salts['salts'].iloc[i].split()[2])
            cations.append(first + ' ' + second)
            
        #these strings have two word anions
        elif("sulfate" in salts['salts'].iloc[i] or\
        "phosphate" in salts['salts'].iloc[i] or\
        "phosphonate" in salts['salts'].iloc[i] or\
        "carbonate" in salts['salts'].iloc[i]):
            first = salts['salts'].iloc[i].split()[1]
            second = salts['salts'].iloc[i].split()[2]
            cations.append(salts['salts'].iloc[i].split()[0])
            anions.append(first + ' ' + second)
        elif("bis(trifluoromethylsulfonyl)imide" in salts['salts'].iloc[i]): 
            #this string contains 2 word cations
            first = salts['salts'].iloc[i].split()[0]
            second = salts['salts'].iloc[i].split()[1]
            third = salts['salts'].iloc[i].split()[2]
            cations.append(first + ' ' + second)
            anions.append(third)
        else:
            print(salts['salts'].iloc[i])
    elif len(salts['salts'].iloc[i].split()) == 4:
        #this particular string block contains (1:1) at end of name
        if("1,1,2,3,3,3-hexafluoro-1-propanesulfonate" in salts['salts'].iloc[i]):
            first = salts['salts'].iloc[i].split()[0]
            second = salts['salts'].iloc[i].split()[1]
            cations.append(first + ' ' + second)
            anions.append(salts['salts'].iloc[i].split()[2])
        else:
            #and two word anion
            first = salts['salts'].iloc[i].split()[1]
            second = salts['salts'].iloc[i].split()[2]
            anions.append(first + ' ' + second)
            cations.append(salts['salts'].iloc[i].split()[0])
    elif("2-aminoethanol-2-hydroxypropanoate" in salts['salts'].iloc[i]):
        #one of the ilthermo salts is missing a space between cation/anion
        anions.append("2-hydroxypropanoate")
        cations.append("2-aminoethanol")
    elif len(salts['salts'].iloc[i].split()) == 5:
        if("bis[(trifluoromethyl)sulfonyl]imide" in salts['salts'].iloc[i]):
            anions.append("bis(trifluoromethylsulfonyl)imide")
            first = salts['salts'].iloc[i].split()[0]
            second = salts['salts'].iloc[i].split()[1]
            third = salts['salts'].iloc[i].split()[2]
            fourth = salts['salts'].iloc[i].split()[3]
            cations.append(first + ' ' + second + ' ' + third + ' ' + fourth)
        if("trifluoro(perfluoropropyl)borate" in salts['salts'].iloc[i]):
            anions.append("trifluoro(perfluoropropyl)borate")
            cations.append("N,N,N-triethyl-2-methoxyethan-1-aminium")    
    else:
        print(salts['salts'].iloc[i])
anions = pd.DataFrame(anions)
anions = anions[0].unique()
cations = pd.DataFrame(cations)
cations = cations[0].unique()

In [231]:
df = pd.read_csv("../salty/data/viscosity_full.csv")
driver = webdriver.Chrome()
driver.get("http://www.chemspider.com/")
time.sleep(5)

In [232]:
ions = anions
timeout = 15
for i in range(len(ions)):
    ion = ions[i]
    ion_df = pd.read_csv("../salty/data/anionInfo.csv")
    if any(ion in s for s in ion_df.anion):
        pass
    else:
        try:
            search = driver.find_element_by_xpath("//*[@id=\"ctl00_ctl00_qs_query\"]")
            search.clear()
            search.send_keys(ion)
            submit = driver.find_element_by_xpath("//*[@id=\"ctl00_ctl00_search_btn\"]")
            submit.click()
            time.sleep(5)
            try:
                moreDetails = driver.find_element_by_xpath("//*[@id=\"ctl00"\
                "_ctl00_ContentSection_ContentPlaceHolder1_RecordViewDetails"\
                "_rptDetailsView_ctl00_structureHead\"]/div[2]/div/h2/a/span")
                moreDetails.click()
                time.sleep(2)
                smiles = driver.find_element_by_xpath("//*[@id=\"ctl00_ctl00"\
                    "_ContentSection_ContentPlaceHolder1_RecordViewDetails"\
                    "_rptDetailsView_ctl00_moreDetails_WrapControl2\"]")
                anions_smiles.append([ion, smiles.text])
                print(ion, smiles.text)
                time.sleep(3)
            except:
                try:
                    firstSearchResult= driver.find_element_by_xpath("//*[@id=\"ctl00_"\
                        "ctl00_ContentSection_ContentPlaceHolder1_ResultViewControl1_grid"\
                        "_GridView1\"]/tbody/tr[1]/td[1]/a")
                    firstSearchResult.click()
                except:
                    partialMatch = driver.find_element_by_xpath("//*[@id=\"ctl00_ctl00_"\
                        "ContentSection_ContentPlaceHolder1_ResultViewControl1_ResultStatement"\
                        "Control1_plhMessage\"]/a")
                    partialMatch.click()
                    time.sleep(6)
                    firstSearchResult= driver.find_element_by_xpath("//*[@id=\"ctl00_"\
                        "ctl00_ContentSection_ContentPlaceHolder1_ResultViewControl1_grid"\
                        "_GridView1\"]/tbody/tr[1]/td[1]/a")
                    firstSearchResult.click()
                time.sleep(5)
                moreDetails = driver.find_element_by_xpath("//*[@id=\"ctl00"\
                    "_ctl00_ContentSection_ContentPlaceHolder1_RecordViewDetails"\
                    "_rptDetailsView_ctl00_structureHead\"]/div[2]/div/h2/a/span")
                moreDetails.click()
                time.sleep(2)
                smiles = driver.find_element_by_xpath("//*[@id=\"ctl00_ctl00"\
                    "_ContentSection_ContentPlaceHolder1_RecordViewDetails"\
                    "_rptDetailsView_ctl00_moreDetails_WrapControl2\"]")
                anions_smiles.append([ion, smiles.text])
                print(ion, smiles.text)
                time.sleep(3)
        except:
            time.sleep(10)
            print("SMILES not found for %s" % ion)

butyl sulfate CCCCOS(=O)(=O)[O-]
hydrogen sulfate OS(=O)(=O)[O-]
methyl sulfate COS(=O)(=O)O
SMILES not found for bis 1,1,1-trifluoro-N-((trifluoromethyl)sulfonyl)methanesulfonamide
butyl phosphonate CCCCOP(=O)OCCCC
ethyl phosphonate CCO[P@@H](=O)OCC
octyl phosphonate CCCCCCCCOP(=O)OCCCCCCCC
hydrogen carbonate C(=O)(O)[O-]
dihydrogen phosphate OP(=O)(O)[O-]
dodecyl sulfate CCCCCCCCCCCCOS(=O)(=O)[O-]
2-hydroxypropanoate CC(C(=O)[O-])O
SMILES not found for 3-butyl-1-methyl-, 1,1,2,3,3,3-hexafluoro-1-propanesulfonate
methyl phosphonate CP(=O)(O)O
2-(2-methoxyethoxy)ethyl sulfate COCCOCCOS(=O)(=O)[O-]
3-((1,3-dihydroxy-2-(hydroxymethyl)propan-2-yl)amino)-2-hydroxypropanesulfonate C(C(CS(=O)(=O)O)O)NC(CO)(CO)CO
2,2,2-trifluoroacetate CC(C)C(C(=O)NC(CCCC[NH3+])C(=O)Nc1cc2c(c3c1cccc3)C(CN2C(=O)/C=C/c4ccc(cc4)OC)CCl)NC(=O)CCOCCOCCOCCOCCNC(=O)CBr.C(=O)(C(F)(F)F)[O-]
SMILES not found for trifluoro(perfluorobutyl)borate
SMILES not found for trifluoro(perfluoroethyl)borate
SMILES not found for tri

In [253]:
anion_df = pd.DataFrame(anions_smiles, columns=["anion","smiles"])
frames = [ion_df, anion_df]
result = pd.concat(frames)

,anion,smiles
0,tetrafluoroborate,[B-](F)(F)(F)F
1,hexafluorophosphate,F[P-](F)(F)(F)(F)F
2,tetracyanoborate,N#C[B-](C#N)(C#N)C#N
3,bis(trifluoromethylsulfonyl)imide,C(F)(F)(F)S(=O)(=O)NS(=O)(=O)C(F)(F)F
4,dicyanamide,C(#N)NC#N
5,trifluoromethanesulfonate,C(F)(F)(F)S(=O)(=O)[O-]
6,methanesulfonate,CS(=O)(=O)[O-]
7,tetrachloroferrate,Cl[Fe-](Cl)(Cl)Cl
8,methylsulfate,COS(=O)(=O)[O-]
9,triflate,C(F)(F)(F)S(=O)(=O)[O-]


In [254]:
pd.DataFrame.to_csv(result, path_or_buf="../salty/data/anionInfo.csv", index=False)

In [59]:
driver = webdriver.Chrome()
driver.get("http://ilthermo.boulder.nist.gov/")
openForm = driver.find_element_by_id("sbutton_label")
openForm.click()
time.sleep(2)
keyword = driver.find_element_by_id("keyw")
keyword.clear()
keyword.send_keys("viscosity")
chemicalFormula = driver.find_element_by_id("cmp")
chemicalFormula.clear()
propert = driver.find_element_by_id("prp")
propert.send_keys("viscosity")
numCompounds = driver.find_element_by_id("ncmp")
numCompounds.send_keys("1")
submit = driver.find_element_by_id("submbutt_label")
submit.click()
time.sleep(10)
salt_list=[]
for page in range(200):
    experimentsOnPage = driver.find_elements_by_xpath("//*[@id\
    [starts-with(., \"dsgrid-row\")]]")
    for thisExperiment in range(len(experimentsOnPage)):
        experimentsOnPage[thisExperiment].click()
        time.sleep(1)
        saltName = driver.find_element_by_xpath("//*[@id=\
            \"dscomp_pane\"]/table/tr[2]/td[2]")
        for dataRow in range(2,200):
            try:
                Temp = driver.find_element_by_xpath("//*[@id=\
                    \"dsdata_pane\"]/table[2]/tr[%s]/td[1]" %dataRow)
                Pres = driver.find_element_by_xpath("//*[@id=\
                    \"dsdata_pane\"]/table[2]/tr[%s]/td[2]" %dataRow)
                Valv = driver.find_element_by_xpath("//*[@id=\
                    \"dsdata_pane\"]/table[2]/tr[%s]/td[3]" %dataRow)
                salt_list.append([Temp.text, Pres.text, Valv.text,\
                                 saltName.text])
                print(Temp.text, Pres.text, Valv.text, saltName.text)
            except:
                break
    nextButton = driver.find_element_by_xpath("//*[@id=\
        \"dsgrid\"]/div[4]/div/div[2]/span[3]")
    nextButton.click()  
    time.sleep(2)

283.09 101.325 0.976 ± 0.067 trihexyl(tetradecyl)phosphonium tris(pentafluoroethyl)trifluorophosphate
283.09 101.325 0.977 ± 0.067 trihexyl(tetradecyl)phosphonium tris(pentafluoroethyl)trifluorophosphate
283.1 101.325 0.99 ± 0.068 trihexyl(tetradecyl)phosphonium tris(pentafluoroethyl)trifluorophosphate
283.1 101.325 0.991 ± 0.069 trihexyl(tetradecyl)phosphonium tris(pentafluoroethyl)trifluorophosphate
283.1 101.325 0.993 ± 0.069 trihexyl(tetradecyl)phosphonium tris(pentafluoroethyl)trifluorophosphate
288.13 101.325 0.693 ± 0.045 trihexyl(tetradecyl)phosphonium tris(pentafluoroethyl)trifluorophosphate
288.13 101.325 0.694 ± 0.045 trihexyl(tetradecyl)phosphonium tris(pentafluoroethyl)trifluorophosphate
288.13 101.325 0.698 ± 0.045 trihexyl(tetradecyl)phosphonium tris(pentafluoroethyl)trifluorophosphate
288.13 101.325 0.699 ± 0.045 trihexyl(tetradecyl)phosphonium tris(pentafluoroethyl)trifluorophosphate
293.07 101.325 0.503 ± 0.03 trihexyl(tetradecyl)phosphonium tris(pentafluoroethyl)trif

333.15 101.325 0.0639 ± 0.0026 trihexyl(tetradecyl)phosphonium tris(pentafluoroethyl)trifluorophosphate
333.15 101.325 0.0642 ± 0.0026 trihexyl(tetradecyl)phosphonium tris(pentafluoroethyl)trifluorophosphate
333.15 101.325 0.0643 ± 0.0026 trihexyl(tetradecyl)phosphonium tris(pentafluoroethyl)trifluorophosphate
333.15 101.325 0.0645 ± 0.0026 trihexyl(tetradecyl)phosphonium tris(pentafluoroethyl)trifluorophosphate
333.15 101.325 0.0645 ± 0.0026 trihexyl(tetradecyl)phosphonium tris(pentafluoroethyl)trifluorophosphate
333.15 101.325 0.0646 ± 0.0027 trihexyl(tetradecyl)phosphonium tris(pentafluoroethyl)trifluorophosphate
333.15 101.325 0.0646 ± 0.0027 trihexyl(tetradecyl)phosphonium tris(pentafluoroethyl)trifluorophosphate
333.15 101.325 0.0646 ± 0.0027 trihexyl(tetradecyl)phosphonium tris(pentafluoroethyl)trifluorophosphate
333.15 101.325 0.0648 ± 0.0027 trihexyl(tetradecyl)phosphonium tris(pentafluoroethyl)trifluorophosphate
333.15 101.325 0.065 ± 0.0027 trihexyl(tetradecyl)phosphonium tr

273.15 100 2.289 ± 0.057 1-methyl-3-octylimidazolium tetrafluoroborate
273.15 100 2.291 ± 0.057 1-methyl-3-octylimidazolium tetrafluoroborate
278.15 100 1.477 ± 0.037 1-methyl-3-octylimidazolium tetrafluoroborate
278.15 100 1.478 ± 0.037 1-methyl-3-octylimidazolium tetrafluoroborate
283.15 100 0.982 ± 0.025 1-methyl-3-octylimidazolium tetrafluoroborate
283.15 100 0.982 ± 0.025 1-methyl-3-octylimidazolium tetrafluoroborate
288.15 100 0.673 ± 0.017 1-methyl-3-octylimidazolium tetrafluoroborate
288.15 100 0.673 ± 0.017 1-methyl-3-octylimidazolium tetrafluoroborate
293.15 100 0.473 ± 0.012 1-methyl-3-octylimidazolium tetrafluoroborate
293.15 100 0.473 ± 0.012 1-methyl-3-octylimidazolium tetrafluoroborate
298.15 100 0.341 ± 0.009 1-methyl-3-octylimidazolium tetrafluoroborate
298.15 11200 0.393 ± 0.01 1-methyl-3-octylimidazolium tetrafluoroborate
298.15 26200 0.476 ± 0.012 1-methyl-3-octylimidazolium tetrafluoroborate
298.15 50500 0.638 ± 0.016 1-methyl-3-octylimidazolium tetrafluoroborate
2

323.15 100 0.0741 ± 0.0063 1-butyl-3-methylimidazolium hexafluorophosphate
323.15 100 0.0749 ± 0.0065 1-butyl-3-methylimidazolium hexafluorophosphate
323.15 1300 0.0749 ± 0.0062 1-butyl-3-methylimidazolium hexafluorophosphate
323.15 3100 0.0769 ± 0.0065 1-butyl-3-methylimidazolium hexafluorophosphate
323.15 5600 0.0782 ± 0.0067 1-butyl-3-methylimidazolium hexafluorophosphate
323.15 10700 0.0827 ± 0.0074 1-butyl-3-methylimidazolium hexafluorophosphate
323.15 25800 0.099 ± 0.011 1-butyl-3-methylimidazolium hexafluorophosphate
323.15 50800 0.128 ± 0.017 1-butyl-3-methylimidazolium hexafluorophosphate
323.15 75800 0.173 ± 0.029 1-butyl-3-methylimidazolium hexafluorophosphate
323.15 100700 0.224 ± 0.046 1-butyl-3-methylimidazolium hexafluorophosphate
323.15 125500 0.281 ± 0.066 1-butyl-3-methylimidazolium hexafluorophosphate
323.15 150400 0.37 ± 0.1 1-butyl-3-methylimidazolium hexafluorophosphate
323.15 175200 0.48 ± 0.15 1-butyl-3-methylimidazolium hexafluorophosphate
323.15 200000 0.62 ± 

278.15 100 0.1404 ± 0.0066 1-butyl-3-methylimidazolium bis(trifluoromethylsulfonyl)imide
278.15 100 0.1404 ± 0.0066 1-butyl-3-methylimidazolium bis(trifluoromethylsulfonyl)imide
283.15 100 0.1061 ± 0.0044 1-butyl-3-methylimidazolium bis(trifluoromethylsulfonyl)imide
283.15 24300 0.1468 ± 0.0069 1-butyl-3-methylimidazolium bis(trifluoromethylsulfonyl)imide
283.15 49900 0.206 ± 0.011 1-butyl-3-methylimidazolium bis(trifluoromethylsulfonyl)imide
283.15 74100 0.28 ± 0.018 1-butyl-3-methylimidazolium bis(trifluoromethylsulfonyl)imide
288.15 100 0.0816 ± 0.003 1-butyl-3-methylimidazolium bis(trifluoromethylsulfonyl)imide
288.15 100 0.0816 ± 0.003 1-butyl-3-methylimidazolium bis(trifluoromethylsulfonyl)imide
293.15 100 0.0638 ± 0.0022 1-butyl-3-methylimidazolium bis(trifluoromethylsulfonyl)imide
293.15 100 0.0639 ± 0.0022 1-butyl-3-methylimidazolium bis(trifluoromethylsulfonyl)imide
298.15 100 0.0509 ± 0.0017 1-butyl-3-methylimidazolium bis(trifluoromethylsulfonyl)imide
298.15 100 0.051 ± 0.0

...

328.15 101 0.0386 ± 0.00073 1-ethylpyridinium ethyl sulfate
298.1 101.325 0.032 ± 0.001 ethylammonium formate
298.1 101.325 0.225 ± 0.014 2-methylpropylammonium formate
298.1 101.325 0.803 ± 0.091 ethylammonium lactate
298.1 101.325 0.075 ± 0.003 ethylammonium propionate
298.1 101.325 0.078 ± 0.003 pentylammonium formate
298.1 101.325 0.229 ± 0.015 2-methylbutylammonium formate
298.1 101.325 0.208 ± 0.013 ethylammonium butyrate
298.1 101.325 1.2 ± 0.16 ethylammonium glycolate
298.1 101.325 0.032 ± 0.001 ethylammonium nitrate
298.1 101.325 1.324 ± 0.18 2-aminoethanol-2-hydroxypropanoate
298.1 101.325 0.128 ± 0.006 ethylammonium hydrogen sulfate
298.1 101.325 0.854 ± 0.099 2-hydroxypropanaminium formate
298.1 101.325 0.017 ± 0.001 methanaminium formate
298.1 101.325 0.07 ± 0.003 butylammonium formate
298.1 101.325 0.113 ± 0.005 ethanolammonium nitrate
294.15 101.325 0.13 ± 0.02 1-butyl-3-methylimidazolium tetrafluoroborate
298.15 101.325 0.682 ± 0.064 1-octyl-3-methylimidazolium hexafluo

298.1 101.325 0.091 ± 0.005 N,N,N-triethyl-2-methoxyethan-1-aminium trifluoro(perfluoropropyl)borate Chemical Formula: C12H22BF10NO
298.1 101.325 0.085 ± 0.004 N,N,N-triethyl-2-methoxyethan-1-aminium bis((trifluoromethyl)sulfonyl)amide
298.15 101 0.1954 ± 0.0072 1-hexyl-3-methylimidazolium tetrafluoroborate
298.1 101.325 0.091 ± 0.005 N,N,N-triethyl-2-methoxyethan-1-aminium trifluoro(perfluoropropyl)borate Chemical Formula: C12H22BF10NO
298.1 101.325 0.085 ± 0.004 N,N,N-triethyl-2-methoxyethan-1-aminium bis((trifluoromethyl)sulfonyl)amide
298.15 101 0.1954 ± 0.0072 1-hexyl-3-methylimidazolium tetrafluoroborate
298.1 101.325 0.091 ± 0.005 N,N,N-triethyl-2-methoxyethan-1-aminium trifluoro(perfluoropropyl)borate Chemical Formula: C12H22BF10NO
298.1 101.325 0.085 ± 0.004 N,N,N-triethyl-2-methoxyethan-1-aminium bis((trifluoromethyl)sulfonyl)amide
298.15 101 0.1954 ± 0.0072 1-hexyl-3-methylimidazolium tetrafluoroborate
298.1 101.325 0.091 ± 0.005 N,N,N-triethyl-2-methoxyethan-1-aminium trifl

298.15 101 0.1954 ± 0.0072 1-hexyl-3-methylimidazolium tetrafluoroborate
298.1 101.325 0.091 ± 0.005 N,N,N-triethyl-2-methoxyethan-1-aminium trifluoro(perfluoropropyl)borate Chemical Formula: C12H22BF10NO
298.1 101.325 0.085 ± 0.004 N,N,N-triethyl-2-methoxyethan-1-aminium bis((trifluoromethyl)sulfonyl)amide
298.15 101 0.1954 ± 0.0072 1-hexyl-3-methylimidazolium tetrafluoroborate
298.1 101.325 0.091 ± 0.005 N,N,N-triethyl-2-methoxyethan-1-aminium trifluoro(perfluoropropyl)borate Chemical Formula: C12H22BF10NO
298.1 101.325 0.085 ± 0.004 N,N,N-triethyl-2-methoxyethan-1-aminium bis((trifluoromethyl)sulfonyl)amide
298.15 101 0.1954 ± 0.0072 1-hexyl-3-methylimidazolium tetrafluoroborate
298.1 101.325 0.091 ± 0.005 N,N,N-triethyl-2-methoxyethan-1-aminium trifluoro(perfluoropropyl)borate Chemical Formula: C12H22BF10NO
298.1 101.325 0.085 ± 0.004 N,N,N-triethyl-2-methoxyethan-1-aminium bis((trifluoromethyl)sulfonyl)amide
298.15 101 0.1954 ± 0.0072 1-hexyl-3-methylimidazolium tetrafluoroborate


298.1 101.325 0.085 ± 0.004 N,N,N-triethyl-2-methoxyethan-1-aminium bis((trifluoromethyl)sulfonyl)amide
298.15 101 0.1954 ± 0.0072 1-hexyl-3-methylimidazolium tetrafluoroborate
298.1 101.325 0.091 ± 0.005 N,N,N-triethyl-2-methoxyethan-1-aminium trifluoro(perfluoropropyl)borate Chemical Formula: C12H22BF10NO
298.1 101.325 0.085 ± 0.004 N,N,N-triethyl-2-methoxyethan-1-aminium bis((trifluoromethyl)sulfonyl)amide
298.15 101 0.1954 ± 0.0072 1-hexyl-3-methylimidazolium tetrafluoroborate
298.1 101.325 0.091 ± 0.005 N,N,N-triethyl-2-methoxyethan-1-aminium trifluoro(perfluoropropyl)borate Chemical Formula: C12H22BF10NO
298.1 101.325 0.085 ± 0.004 N,N,N-triethyl-2-methoxyethan-1-aminium bis((trifluoromethyl)sulfonyl)amide
298.15 101 0.1954 ± 0.0072 1-hexyl-3-methylimidazolium tetrafluoroborate
298.1 101.325 0.091 ± 0.005 N,N,N-triethyl-2-methoxyethan-1-aminium trifluoro(perfluoropropyl)borate Chemical Formula: C12H22BF10NO
298.1 101.325 0.085 ± 0.004 N,N,N-triethyl-2-methoxyethan-1-aminium bis((

298.1 101.325 0.091 ± 0.005 N,N,N-triethyl-2-methoxyethan-1-aminium trifluoro(perfluoropropyl)borate Chemical Formula: C12H22BF10NO
298.1 101.325 0.085 ± 0.004 N,N,N-triethyl-2-methoxyethan-1-aminium bis((trifluoromethyl)sulfonyl)amide
298.15 101 0.1954 ± 0.0072 1-hexyl-3-methylimidazolium tetrafluoroborate
298.1 101.325 0.091 ± 0.005 N,N,N-triethyl-2-methoxyethan-1-aminium trifluoro(perfluoropropyl)borate Chemical Formula: C12H22BF10NO
298.1 101.325 0.085 ± 0.004 N,N,N-triethyl-2-methoxyethan-1-aminium bis((trifluoromethyl)sulfonyl)amide
298.15 101 0.1954 ± 0.0072 1-hexyl-3-methylimidazolium tetrafluoroborate
298.1 101.325 0.091 ± 0.005 N,N,N-triethyl-2-methoxyethan-1-aminium trifluoro(perfluoropropyl)borate Chemical Formula: C12H22BF10NO
298.1 101.325 0.085 ± 0.004 N,N,N-triethyl-2-methoxyethan-1-aminium bis((trifluoromethyl)sulfonyl)amide
298.15 101 0.1954 ± 0.0072 1-hexyl-3-methylimidazolium tetrafluoroborate
298.1 101.325 0.091 ± 0.005 N,N,N-triethyl-2-methoxyethan-1-aminium trifl

In [61]:
df = pd.DataFrame(salt_list,columns=["Temperature, K","Pressure, kPa","Viscosity, Pas","salt_name"])
pd.DataFrame.to_csv(df, path_or_buf="../salty/data/viscosity_full.csv", index=False)